In [1]:
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import numpy as np
import matplotlib as mpl
import cartopy.feature as cfeature
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText
import matplotlib.cm as cm
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import glob
import matplotlib.colors as colors
import intake
import dask
from scipy.spatial import cKDTree
import networkx as nx
import psyplot.project as psy
import pandas as pd
from scipy.stats import pearsonr
from pandas.plotting import lag_plot
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
import warnings
from statsmodels.tsa.stattools import grangercausalitytests
import scipy.stats
from scipy import stats

dask.config.set({"array.slicing.split_large_chunks": True}) 

In [2]:
eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
eerie_cat["dkrz.disk.model-output.icon-esm-er.eerie-control-1950.atmos.gr025"]

gr025:
  args:
    path: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-control-1950/atmos/gr025/main.yaml
  description: This catalog contains atmospheric EERIE ICON-ESM-ER eerie-control-1950
    output on 0.25deg grid available at DKRZ disk.
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    catalog_dir: https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/dkrz/disk/model-output/icon-esm-er/eerie-control-1950/atmos


In [3]:
eerie_dkrz=eerie_cat["dkrz"]

In [4]:
eerie_dkrz_disk=eerie_dkrz["disk"]
#for col in eerie_dkrz_disk:
#    print(f"Description of {col}:")
#    print(eerie_dkrz_disk[col].describe()["description"])

In [5]:
cat=eerie_dkrz_disk["model-output"]["icon-esm-er"]

In [6]:
searchdict=dict(
    model="ICON",
    realm="atmos",
    exp="eerie-control-1950",
    var="pr",
    frequency="daily"
)
subcat=cat
for v in searchdict.values():
    subcat=subcat.search(v)
list(subcat)

['eerie-control-1950.atmos.gr025.2d_daily_max',
 'eerie-control-1950.atmos.gr025.2d_daily_mean',
 'eerie-control-1950.atmos.gr025.2d_daily_min',
 'eerie-control-1950.atmos.gr025.plev19_daily_mean',
 'eerie-control-1950.atmos.native.2d_daily_max',
 'eerie-control-1950.atmos.native.2d_daily_mean',
 'eerie-control-1950.atmos.native.2d_daily_min',
 'eerie-control-1950.atmos.native.model-level_daily_mean_1',
 'eerie-control-1950.atmos.native.model-level_daily_mean_2',
 'eerie-control-1950.atmos.native.plev19_daily_mean']

In [7]:
#EKE
ds=cat["eerie-control-1950.ocean.gr025.2d_daily_mean"].to_dask()
ds

<xarray.Dataset>
Dimensions:                              (time: 13695, lat: 721, lon: 1440,
                                          lev: 1, depth: 1)
Coordinates:
  * depth                                (depth) float64 1.0
  * lat                                  (lat) float64 -90.0 -89.75 ... 90.0
  * lev                                  (lev) float64 0.0
  * lon                                  (lon) float64 0.0 0.25 ... 359.5 359.8
  * time                                 (time) datetime64[ns] 2002-01-01T23:...
Data variables: (12/30)
    Wind_Speed_10m                       (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    atmos_fluxes_FrshFlux_Evaporation    (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    atmos_fluxes_FrshFlux_Precipitation  (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    atmos_fluxes_FrshFlux_Runoff         (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    atmos_fluxes_FrshFlux_SnowFall       (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    atmos_fluxes_HeatFlux_Latent         (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    ...                                   ...
    so                                   (time, depth, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    ssh                                  (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    stretch_c                            (time, lat, lon) float32 dask.array<chunksize=(32, 721, 1440), meta=np.ndarray>
    to                                   (time, depth, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    u                                    (time, depth, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
    v                                    (time, depth, lat, lon) float32 dask.array<chunksize=(32, 1, 721, 1440), meta=np.ndarray>
Attributes: (12/31)
    CDI:                       Climate Data Interface version 2.2.4 (https://...
    CDO:                       Climate Data Operators version 2.2.2 (https://...
    Conventions:               CF-1.6
    DOKU_License:              CC BY 4.0
    DOKU_Name:                 EERIE ICON-ESM-ER eerie-control-1950 run
    DOKU_authors:              Putrasahan, D.; Kröger, J.; Wachsmann, F.
    ...                        ...
    simulation_id:             erc1011
    source:                    git@gitlab.dkrz.de:icon/icon-mpim.git@45022778...
    source_id:                 ICON-ESM
    source_type:               AOGCM
    time_reduction:            mean
    title:                     ICON simulation

In [20]:

#make the mean
step=1
u_mean=np.mean(ds.u[:365,0,::step,::step], axis=0) #mean across axis=0, ovvero il tempo
v_mean=np.mean(ds.v[:365,0,::step,::step], axis=0)

#calculate eddy velocities
u_eddy=(ds.u[:365,0,::step,::step].values)-(u_mean[:,:].values)
v_eddy=(ds.v[:365,0,::step,::step].values)-(v_mean[:,:].values)

#calculate eke
eke_s=(((u_eddy)**2)+((v_eddy)**2))/2

In [11]:
atm=cat['eerie-control-1950.atmos.gr025.2d_daily_mean'].to_dask()

In [21]:
pr=atm['pr'][:365,:,:].values
#np.save('atmfrshflux2002.npy', frshflux)
pr_s=atm['pr'][:365,::step,::step].values


In [22]:
pr_s.shape

(365, 721, 1440)

In [2]:
#TO RE-LOAD
# Read the graph from the GraphML file
G_loaded = nx.read_graphml("correlation_graph_2002_step10.graphml")

# Convert node strings back to tuples---> THIS IS THE NETWORK
G = nx.relabel_nodes(G_loaded, {node: eval(node) for node in G_loaded.nodes()})

# Now G_original_labels should have nodes represented as tuples
#pos = {node: (node[1], node[0]) for node in G_original_labels.nodes()}



<font size=5> The cell below imports a parallelising library:

In [23]:
from concurrent.futures import ProcessPoolExecutor
n_cpu = 64
#max number of CPUs I can use is the number of latitudes we have

In [ ]:
#PROVA2


latitudes = ds.lat.values[::step]
longitudes = ds.lon.values[::step]

eke_downsized = eke_s
frshflux_downsized = pr_s

# Get the dimensions of the downsized arrays
num_time_steps, num_latitudes, num_longitudes = eke_downsized.shape



# Set the correlation threshold
correlation_threshold = 0.5

def parallel_func(lat):
    
    # Create a graph
    S = nx.Graph()
    
    for lon in range(num_longitudes):
        if not np.any(np.isnan(eke_downsized[:, lat, lon])):
            # Extract time series at the current 'eke' position
            time_series_eke = eke_downsized[:, lat, lon]

            # Iterate over all positions (lat, lon) for 'frshflux'
            for lat_frshflux in range(num_latitudes):
                for lon_frshflux in range(num_longitudes):
                    if not np.any(np.isnan(frshflux_downsized[:, lat_frshflux, lon_frshflux])):
                        # Extract time series at the current 'frshflux' position
                        time_series_frshflux = frshflux_downsized[:, lat_frshflux, lon_frshflux]


                        # Check for NaN values
                        if not (np.any(np.isnan(time_series_eke)) or np.any(np.isnan(time_series_frshflux))):
                            # Calculate Pearson correlation coefficient
                            correlation, _ = pearsonr(time_series_eke, time_series_frshflux)

                            # Check if correlation is above the threshold
                            if abs(correlation) > correlation_threshold:
                                # Add nodes to the graph
                                node_eke = (ds.lat.values[::step][lat], ds.lon.values[::step][lon], 'eke')
                                node_frshflux = (ds.lat.values[::step][lat_frshflux], ds.lon.values[::step][lon_frshflux], 'frshflux')
                                S.add_node(node_eke)
                                S.add_node(node_frshflux)

                                # Add weighted edge with correlation coefficient as weight
                                S.add_weighted_edges_from([(node_eke, node_frshflux, correlation)])

    return S



with ProcessPoolExecutor(max_workers=n_cpu) as executor:
    # Call the function
    S_list = list(executor.map(parallel_func, range(num_latitudes))) 


    

#SAVE:

# Convert node tuples to strings before saving
#S_str_labels = nx.relabel_nodes(S, {node: str(node) for node in S.nodes()})

# Save the graph to a GraphML file
#nx.write_graphml(S_str_labels, "test_eke_atmrain_correlation_graph_2002_step10.graphml")

<font size=5> The network now iterates over each latitude on a different cpu, and creates different graphs (S_list), that are then joined with nx.compose. executor.map calls a function and the variable over which to iterate, which is range(num_latitudes) (above)

In [15]:
W=nx.Graph()
for i in range(len(S_list)):
    W=nx.compose(W,S_list[i])

In [17]:
W.number_of_edges()

2778

In [18]:
W.edges(data=True)

EdgeDataView([((-77.5, 165.0, 'eke'), (-5.0, 260.0, 'frshflux'), {'weight': 0.5123810776678178}), ((-77.5, 165.0, 'eke'), (2.5, 252.5, 'frshflux'), {'weight': 0.543302778688015}), ((-77.5, 165.0, 'eke'), (2.5, 255.0, 'frshflux'), {'weight': 0.5311500600326103}), ((-77.5, 165.0, 'eke'), (2.5, 260.0, 'frshflux'), {'weight': 0.5076545047969486}), ((2.5, 252.5, 'frshflux'), (-72.5, 302.5, 'eke'), {'weight': 0.5018597633205455}), ((2.5, 252.5, 'frshflux'), (-27.5, 227.5, 'eke'), {'weight': 0.5443669324095076}), ((2.5, 252.5, 'frshflux'), (-5.0, 162.5, 'eke'), {'weight': 0.6119322836836385}), ((2.5, 252.5, 'frshflux'), (25.0, 197.5, 'eke'), {'weight': 0.5283225363889802}), ((2.5, 252.5, 'frshflux'), (75.0, 210.0, 'eke'), {'weight': 0.5012569591167747}), ((2.5, 260.0, 'frshflux'), (-27.5, 227.5, 'eke'), {'weight': 0.5155594843545247}), ((2.5, 260.0, 'frshflux'), (77.5, 17.5, 'eke'), {'weight': 0.5136949409644}), ((-77.5, 167.5, 'eke'), (-80.0, 172.5, 'frshflux'), {'weight': 0.5370370104707975